<a href="https://colab.research.google.com/github/Yassmina-Abdo/realtime-head-pose-estimation/blob/main/Notebooks/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Liberaries

In [1]:
import numpy as np
import pickle
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR, SVR,SVC
from sklearn.preprocessing import MinMaxScaler,StandardScaler

## Reading the prepared dataset and split it

In [2]:
path_local='../Data/media_with_angles.csv'
path_drive='/content/drive/MyDrive/Head Pose Estimation ML Project/Dataset/Data/data68.csv'
data =pd.read_csv(path_local)
data.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y463,y464,y465,y466,y467,y468,image_name,yaw,pitch,roll
0,247.0,265.0,248.0,254.0,268.0,265.0,253.0,177.0,253.0,256.0,...,292.0,228.0,231.0,234.0,223.0,219.0,image03195,41.555737,-8.044670,-10.008090
1,220.0,211.0,216.0,205.0,210.0,210.0,213.0,163.0,212.0,211.0,...,281.0,222.0,224.0,226.0,212.0,210.0,image03247,-13.392587,-6.686405,5.489456
2,188.0,171.0,188.0,173.0,168.0,171.0,182.0,174.0,182.0,178.0,...,272.0,219.0,223.0,225.0,209.0,203.0,image03214,-76.223580,0.176040,20.775415
3,223.0,222.0,225.0,216.0,223.0,223.0,227.0,157.0,228.0,228.0,...,297.0,213.0,219.0,224.0,197.0,193.0,image03238,-2.343783,-27.163183,1.134514
4,219.0,227.0,223.0,222.0,229.0,230.0,232.0,165.0,234.0,236.0,...,288.0,218.0,221.0,224.0,213.0,210.0,image03236,11.787567,-13.828854,-8.607656


In [3]:
X = data.iloc[:,:-4]
X.head(1)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y459,y460,y461,y462,y463,y464,y465,y466,y467,y468
0,247.0,265.0,248.0,254.0,268.0,265.0,253.0,177.0,253.0,256.0,...,290.0,287.0,290.0,291.0,292.0,228.0,231.0,234.0,223.0,219.0


In [8]:
y = data.iloc[:,-3:]
y.head(1)

,yaw,pitch,roll
0,41.555737,-8.04467,-10.00809


## Scaling

In [6]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(X)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, shuffle=True)

## **Trial #1:**

- MultiOutputRegressor with RandomForestRegressor


> https://www.r-bloggers.com/2021/10/multi-output-regression-using-sklearn/





In [10]:
clf = MultiOutputRegressor(RandomForestRegressor(max_depth=12, random_state=42,n_estimators=300,min_samples_split=4))
clf.fit(x_train, y_train)
clf.score(x_train, y_train, sample_weight=None)

0.9404599820425669

In [11]:
y_pred= clf.predict(x_test)
clf.score(x_test, y_test, sample_weight=None)

0.660921289721109

In [12]:
# save model
with open('../SavedModels/model_Rm.pkl', 'wb') as file:
        pickle.dump(clf, file)

## **Trial #2:**
  - SVM

  - https://machinelearningmastery.com/multi-output-regression-models-with-python/

In [ ]:
# define base model
model = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1,coef0=1)
# define the direct multioutput wrapper model
wrapper = MultiOutputRegressor(model)
wrapper.fit(x_train, y_train)
wrapper.predict(x_test)

In [ ]:
print(wrapper.score(x_train, y_train, sample_weight=None))
print(wrapper.score(x_test, y_test, sample_weight=None))

In [15]:
# save model
with open('../SavedModels/model_svm.pkl', 'wb') as file:
        pickle.dump(wrapper, file)

## **Trial #3:**

- GradientBoostingRegressor

In [16]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(random_state=42,n_estimators=100,learning_rate=0.05)
model = MultiOutputRegressor(estimator=gbr)

In [17]:
model.fit(x_train, y_train)
score = model.score(x_train, y_train)
print("Training score:", score)
ypred = model.predict(x_test)
model.score(x_test, y_test, sample_weight=None)

Training score: 0.98825433442155


0.8576349750525792

In [18]:
# save model
with open('../SavedModels/model_GB.pkl', 'wb') as file:
        pickle.dump(model, file)